# 实验. 基于GCN的致病基因预测
## 实验目的
1. 实现基于Mindspore实现的GCN模型。
2. 使用GCN模型进行致病基因预测，了解网络机器学习模型解决排序问题。

In [13]:
import numpy as np
from scipy.io import loadmat
import mindspore as ms
from mindspore import nn, ops, Tensor
# 固定随机种子
np.random.seed(42)
ms.set_seed(42)

## 1. 装载数据
该部分将三个网络数据进行加载，分别存放在phenotype_network、ppi_network和g_p_network中。其中疾病相似度网phenotype_network的第一列为疾病表型的OMIM ID

In [14]:
# 加载ppi_network，该网络储存了基因与基因之间的相似关系。基因总数为8919，故该网络维度为8919*8919。
ppi = loadmat("ppi_network.mat")
ppi_network = ppi["ppi_network"]
gene_name = ppi["gene_name"]

# 加载phenotype_network，该网络储存了疾病与疾病间的相似度关系。疾病总数为5080。注意第一列为疾病ID，故该网络维度为5080*5081
phenotype = loadmat("phenotype_network.mat")
phenotype_network = phenotype["phenotype_network"]
phenotype_name = phenotype["phenotype_name"]


# 加载g_p_network，该网络储存了基因与疾病间的关联关系，若第i个基因与第j个疾病有关，则该位置元素为1，无关为0。维度为8919*5080。
g_p_network = loadmat("g_p_network.mat")["g_p_network"]

## 2. 获取致病基因集合
### 2.1 返回目标疾病的最相似的5个疾病表型

In [15]:
def getIndexDisease(DiseaseID):
    m = phenotype_network.shape[0]
    line_num = 0
    for i in range(m):
        if phenotype_network[i, 0] == DiseaseID:
            line_num = i
            break
    return line_num


def getSimilarDiseases(DiseaseID, phenotype_network):
    line_num = getIndexDisease(DiseaseID)

    similarity_vec = phenotype_network[line_num, 1:]

    pos = np.argsort(similarity_vec)  # 对similarity_vec排序，不返回排序结果，返回排序值的index
    similarDiseasePos = pos[-5:]  # 从倒数第5个元素开始取到最后。

    similarDiseaseID = phenotype_network[similarDiseasePos, 0]

    return similarDiseaseID, similarDiseasePos

def getDiseaseDescription(DiseaseID):
    line_num = getIndexDisease(DiseaseID)
    return phenotype_name[line_num]


# 设置疾病ID，该ID需要通过phenotype_network的第一列查看。可以单独存储。
DiseaseID = 114480 #114480✔ # 104300
print(f"选择的疾病ID为：{DiseaseID},其对应描述为:{getDiseaseDescription(DiseaseID)[0]}")
# print(getDiseaseDescription(DiseaseID))

similarDiseaseID, similarDiseasePos = getSimilarDiseases(DiseaseID, phenotype_network)
print("相似病理的ID为：")
print(similarDiseaseID)
print("对应的疾病描述依次为：")
for i in range(5):
    print(phenotype_name[similarDiseasePos[i]][0][0])

# print(similarDiseasePos)

选择的疾病ID为：114480,其对应描述为:['BREAST CANCER']
相似病理的ID为：
[155720. 120435. 113705. 176807. 114480.]
对应的疾病描述依次为：
MELANOMA, UVEAL
LYNCH SYNDROME I
BREAST CANCER 1 GENE; BRCA1
PROSTATE CANCER
BREAST CANCER


### 2.2 获取致病基因集合
获取到相应疾病对应的致病基因的集合。

In [16]:
# 获取疾病集合的下标
similarDiseasePos_List = similarDiseasePos.tolist()
DiseaseSetPos = similarDiseasePos_List + [getIndexDisease(DiseaseID)]


# 取集合节点embedding的平均值代表整个致病基因集合
# 提取 DiseaseSetPos 对应的嵌入并求平均
# 获取对应的基因ID
GeneSetPos = []
Gene_nums = ppi_network.shape[0]
for DiseasePos in DiseaseSetPos:
    for i in range(Gene_nums):
        if g_p_network[i,DiseasePos] == 1:
            GeneSetPos.append(i)

# 去重
GeneSetPos = list(set(GeneSetPos))
print("致病基因")
print(GeneSetPos)
# 输出致病基因描述
print("致病基因集合描述：")
for Gene_Pos in GeneSetPos:
    print(f"基因 {Gene_Pos}: {gene_name[Gene_Pos][0][0]}")




致病基因
[1376, 1634, 101, 5447, 1479, 1385, 6733, 3062, 1370]
致病基因集合描述：
基因 1376: RAD51
基因 1634: PTEN
基因 101: BRCA1
基因 5447: RNASEL
基因 1479: NBN
基因 1385: BRCA2
基因 6733: ELAC2
基因 3062: PIK3CA
基因 1370: MSH2


## 3.构造基于GCN的致病基因预测模型
### 3.1 定义GCN模型
使用MindSpore定义一个GCN模型，并创建。

In [ ]:
# 定义 GCN 层
class GCNLayer(nn.Cell):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.weight = nn.Dense(in_channels, out_channels, has_bias=False)

    def construct(self, X, A):
        support = self.weight(X)
        output = ops.matmul(A, support)
        return output

# 定义升级版 GCN 模型
class GCNModel(nn.Cell):
    def __init__(self, hidden_dim=128, out_dim=64, dropout_rate=0.5, feature_dim=64):
        super(GCNModel, self).__init__()
        self.gc1 = GCNLayer(feature_dim, hidden_dim)
        self.gc2 = GCNLayer(hidden_dim, out_dim)
        self.relu = ops.ReLU()
        self.dropout = nn.Dropout(keep_prob=1 - dropout_rate)

    def construct(self, A):
        num_nodes = A.shape[0]
        
        # 生成单位矩阵特征
        X = Tensor(np.eye(num_nodes, dtype=np.float32))  # [num_nodes, num_nodes]
        A_tensor = Tensor(A.astype(np.float32))

         # 第一层h1
        h1 = self.gc1(X, A_tensor)
        h1 = self.relu(h1)
        h1 = self.dropout(h1)
        
        # 第二层h2
        h2 = self.gc2(h1, A_tensor)
        
        return h2
    
# 加载预训练模型
model = GCNModel(hidden_dim=128, out_dim= 256,feature_dim = ppi_network.shape[0])



[WARNING] ME(6912:15156,MainProcess):2025-05-12-16:41:33.226.412 [mindspore\nn\layer\basic.py:176] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


### 3.2 训练GCN模型
基于致病基因集合训练GCN模型，以实现隐形致病基因的预测

In [18]:
# 1) Masked Cross‐Entropy Loss
class MaskedSoftmaxCrossEntropy(nn.Cell):
    def __init__(self):
        super().__init__()
        self.ce = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='none')
        self.reduce_sum = ops.ReduceSum()
        self.cast = ops.Cast()

    def construct(self, logits, labels, mask):
        # logits: [N, 2], labels: [N], mask: [N]
        losses = self.ce(logits, labels)            # [N], 全部节点的 loss
        mask_f = self.cast(mask, losses.dtype)      # 布尔 -> float
        masked = losses * mask_f                    # 未标注位置被乘成 0
        # 平均到“真正有标签”的节点上
        return self.reduce_sum(masked) / self.reduce_sum(mask_f)

# 2) 训练步骤封装
class TrainStep(nn.Cell):
    def __init__(self, network, loss_fn, optimizer):
        super().__init__()
        self.network = network
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.grad_op = ops.GradOperation(get_by_list=True)
        self.params = network.trainable_params()

    def construct(self, A, labels, mask):
        def forward(A, labels, mask):
            logits = self.network(A)
            loss = self.loss_fn(logits, labels, mask)
            return loss

        grad_fn = self.grad_op(forward, self.params)
        grads = grad_fn(A, labels, mask)
        self.optimizer(grads)

        # 返回当前 loss 值用于打印
        return forward(A, labels, mask)

# 3) 整体训练代码
# 假设你已经有 ppi_network, GeneSetPos
num_nodes = ppi_network.shape[0]
# 3.1 重新准备标签 & mask
raw_labels = np.zeros((num_nodes,), dtype=np.int32)
raw_labels[GeneSetPos] = 1
labels = Tensor(raw_labels)
mask = Tensor((raw_labels == 1))

def normalize_adj(adj):
    adj = adj + np.eye(adj.shape[0])  # 添加自环
    d = np.sum(adj, axis=1)
    d_sqrt_inv = np.power(d, -0.5).flatten()
    d_sqrt_inv[np.isinf(d_sqrt_inv)] = 0.0
    d_mat_inv_sqrt = np.diag(d_sqrt_inv)
    adj_normalized = adj.dot(d_mat_inv_sqrt).T.dot(d_mat_inv_sqrt).astype(np.float32)
    return adj_normalized
A_norm = Tensor(normalize_adj(ppi_network).astype(np.float32))

# 3.3 实例化模型、loss、optimizer、train_step
loss_fn   = MaskedSoftmaxCrossEntropy()
optimizer = nn.Adam(model.trainable_params(), learning_rate=1e-4)
train_step = TrainStep(network=model, loss_fn=loss_fn, optimizer=optimizer)
print("Traning begin.")
# 3.4 训练循环
num_epochs = 10
for epoch in range(num_epochs):
    loss = train_step(A_norm, labels, mask)
    print(f"Epoch {epoch:03d}, loss={loss.asnumpy():.4f}")
print("Training completed.")

Traning begin.
Epoch 000, loss=5.5437
Epoch 001, loss=5.5434
Epoch 002, loss=5.5430
Epoch 003, loss=5.5426
Epoch 004, loss=5.5423
Epoch 005, loss=5.5419
Epoch 006, loss=5.5415
Epoch 007, loss=5.5411
Epoch 008, loss=5.5408
Epoch 009, loss=5.5404
Training completed.


## 4. 预测致病基因
### 4.1 计算相似度并排序
首先获得Embeddings结果，然后使用余弦距离计算各个基因与致病基因集合的相似度，并排序。输出Topk个相似基因

In [19]:
# 余弦距离
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    if len(vec1) != len(vec2):
        raise ValueError("向量维度不一致！")
    
    # 计算点积和模长
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0  # 避免除以零
    
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity


# 获取节点嵌入
embeddings_tensor = model(A_norm)
embeddings = embeddings_tensor.asnumpy()

GeneSetPos_embeddings = embeddings[GeneSetPos]  # 提取对应下标的嵌入
GeneSetEmbedding =  np.mean(GeneSetPos_embeddings, axis=0)  # 按行求平均
print(GeneSetEmbedding)


# 初始化存储结果的列表
distances = []
gene_indices = []
# 遍历所有基因（排除致病基因集合中的基因）
for i in range(embeddings.shape[0]):
    if i not in GeneSetPos:  # 排除致病基因集合中的基因
        # 计算当前基因与致病基因集合的余弦距离
        # 余弦距离
        distance = cosine_similarity(embeddings[i],GeneSetEmbedding)
        distances.append(distance)
        gene_indices.append(i)


# 将距离和基因下标组合成一个数组
distance_gene_pairs = list(zip(distances, gene_indices))


# 按余弦相似度从高到低排序（相似度越高，越靠前）
distance_gene_pairs.sort(key=lambda x: x[0], reverse=True)

[-3.35501420e-04  4.75351512e-03 -4.91864630e-04  8.79274565e-04
  3.90439964e-04 -9.60023666e-04  1.23517576e-03  1.65464671e-03
  6.93010690e-04 -4.53583576e-04  4.75402921e-04 -1.19170616e-03
 -1.85371071e-04 -4.95747139e-04 -6.42566243e-04  5.63571171e-04
  6.72685041e-04 -7.21439195e-04 -3.59036610e-04  6.06816102e-05
  5.24536124e-04  1.01719936e-03 -7.88410951e-04 -1.89437342e-04
 -4.01187790e-05 -2.34150662e-04  6.11822179e-04  1.02668440e-04
  4.03551439e-06  3.16550722e-05 -1.27260527e-03 -9.59561439e-04
 -9.34002106e-04  2.20856571e-04  9.90143279e-04  3.57539131e-04
 -1.75870955e-04  2.10420432e-04 -2.31791491e-04 -1.04480787e-04
 -4.21823497e-04  4.89774742e-04  5.35841857e-04 -5.35775558e-04
 -2.98400060e-04 -2.70551915e-04  6.60656311e-04  8.66233750e-05
 -9.58110322e-04 -1.95218890e-04  1.96419563e-03  1.83324228e-04
 -4.30458080e-04  5.55789120e-05  4.52537613e-04 -8.76158811e-05
 -4.24847676e-04  3.38484242e-04 -6.76680182e-04 -4.95379609e-05
  6.79739518e-04  6.56815

### 4.2 输出排序后的top 10基因

In [20]:
# 输出 Top-k 个最相似的基因
k = 10  # 设置 Top-k 的值
top_k_genes = distance_gene_pairs[:k]


print(f"选择的疾病ID为：{DiseaseID},其对应描述为:{getDiseaseDescription(DiseaseID)[0][0]}")
# 打印结果
print(f"Top-{k} 最可能的致病基因为：")
for distance, gene_idx in top_k_genes:
    print(f"基因 {gene_idx}: {gene_name[gene_idx][0][0]} (余弦距离: {distance:.4f})")

选择的疾病ID为：114480,其对应描述为:BREAST CANCER
Top-10 最可能的致病基因为：
基因 125: STAT3 (余弦距离: 0.9669)
基因 163: EP300 (余弦距离: 0.9643)
基因 154: AR (余弦距离: 0.9636)
基因 144: JUN (余弦距离: 0.9631)
基因 157: STAT1 (余弦距离: 0.9617)
基因 1759: CEBPB (余弦距离: 0.9603)
基因 71: PML (余弦距离: 0.9594)
基因 1379: MYC (余弦距离: 0.9584)
基因 85: DAXX (余弦距离: 0.9578)
基因 237: SP1 (余弦距离: 0.9578)
